In [5]:
import pandas as pd


In [2]:
df = pd.read_json('final_data.json')
df

,name,state,country,location
0,Ravi river,Punjab,India,"Ravi river, Punjab, India"
1,Punjab plain,Haryana,India,"Punjab plain, Haryana, India"
2,Jhelum river,Punjab,Pakistan,"Jhelum river, Punjab, Pakistan"
3,Hindustan,Delhi,India,"Hindustan, Delhi, India"
4,Basantar river,Himachal Pradesh,India,"Basantar river, Himachal Pradesh, India"
...,...,...,...,...
812214,Athal,Dadra and Nagar Haveli and Daman and Diu,India,"Athal, Dadra and Nagar Haveli and Daman and Di..."
812218,Karad daman project,Dadra and Nagar Haveli and Daman and Diu,India,"Karad daman project, Dadra and Nagar Haveli an..."
812219,Dapada,Dadra and Nagar Haveli and Daman and Diu,India,"Dapada, Dadra and Nagar Haveli and Daman and D..."
812220,Surangi,Dadra and Nagar Haveli and Daman and Diu,India,"Surangi, Dadra and Nagar Haveli and Daman and ..."


In [3]:
df.drop_duplicates(subset='location', inplace=True, ignore_index=True)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import save_npz, load_npz

In [5]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['location'])

In [6]:
import joblib

tfidf_vectorizer_filename = 'model/tfidf_vectorizer.pkl'
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_filename)

['model/tfidf_vectorizer.pkl']

In [7]:
index_mapping = df['location']
tfidf_matrix_filename = 'model/tfidf_matrix.npz'
index_mapping_filename = 'model/index_mapping.pkl'

# Save the precomputed vectors and metadata
save_npz(tfidf_matrix_filename, tfidf_matrix)
index_mapping.to_pickle(index_mapping_filename)

In [9]:
import joblib
tfidf_matrix_filename = 'model/tfidf_matrix.npz'
index_mapping_filename = 'model/index_mapping.pkl'
tfidf_vectorizer_filename = 'model/tfidf_vectorizer.pkl'

def load_sparse_csr(filename):
    return load_npz(filename)

# Query and Retrieve
def query(input_text, tfidf_matrix_filename, index_mapping_filename, top_n=5):
    # Load precomputed vectors and metadata
    tfidf_matrix = load_sparse_csr(tfidf_matrix_filename)
    index_mapping = pd.read_pickle(index_mapping_filename)
    result = []
    tfidf_vectorizer_new = joblib.load(tfidf_vectorizer_filename)
    # Vectorize the input text
    input_vector = tfidf_vectorizer_new.transform([input_text])
    print(input_vector.shape)
    print(tfidf_matrix.shape)

    # Compute cosine similarity between input text vector and precomputed vectors
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix).flatten()

    # Retrieve top results
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    top_results = index_mapping.loc[top_indices]

    for res in top_results:
        result.append(
            {
                'entity_name': res
            }
        )
    return result


In [10]:
input_text = 'west'
top_results = query(input_text, tfidf_matrix_filename, index_mapping_filename)
print(top_results)

(1, 365180)
(549264, 365180)
[{'entity_name': 'West bengal, West Bengal, India'}, {'entity_name': 'Bamunia west, West Bengal, India'}, {'entity_name': 'West sanabandh, West Bengal, India'}, {'entity_name': 'Khurd, West Bengal, India'}, {'entity_name': 'Ramgarh west midnapor, West Bengal, India'}]
